In [2]:
import torch
import torch.nn as nn

# 定义模型
class HwdrCNN(nn.Module):
    def __init__(self):
        super(HwdrCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) 

        self.activation = nn.ReLU()  # 激活函数
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # 最大池化层

        self.full_conn1 = nn.Linear(64 * 7 * 7, 128)  # 全连接层
        self.full_conn2 = nn.Linear(128, 10)  # 输出层（10 个类别）

    def forward(self, x):
        # conv layer 1
        x = self.conv1(x) 
        x = self.activation(x)
        x = self.pool(x)

        # conv layer 2
        x = self.conv2(x)   
        x = self.activation(x)
        x = self.pool(x) 

        # 展开
        x = x.view(-1, 64 * 7 * 7)

        x = self.full_conn1(x)   
        x = self.activation(x)  

        x = self.full_conn2(x) 
        return x
    

In [22]:
import os
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image


torch.cuda.is_available()
SAVE_MODEL_DIR = "./checkpoints"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
model = HwdrCNN()

# 加载模型
para_path = os.path.join(SAVE_MODEL_DIR, os.listdir(SAVE_MODEL_DIR)[-1])
model.load_state_dict(torch.load(para_path,weights_only=True))

# print(model)

test_data_dir = "test_data"
test_image = os.listdir(test_data_dir)[0]
test_image_oath = os.path.join(test_data_dir, test_image)


transform = transforms.Compose([
    transforms.Resize((28, 28)),  # 调整大小为28x28
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 归一化
])

img = Image.open(test_image_oath).convert('L')  # 灰度图
img.show()

img = transform(img)  # 转换为张量
img = img.unsqueeze(0)  # 增加batch维度


model.eval()
with torch.no_grad():  # 不计算梯度
    output = model(img)

print(output)

probabilities = F.softmax(output, dim=1)  # dim=1表示对每一行进行softmax
for i, prob in enumerate(probabilities[0]):
    print(f'{i:1d} : {prob.item():.4f}')

_, predicted = torch.max(output, 1)  # 获取预测的类别

print(f'预测结果: {predicted.item()}')  # 输出预测的数字


tensor([[  0.4652,  -4.4744,   4.0792,   3.9256,  -4.7411,  -8.8089,  -1.7267,
          -7.2751,   0.4303, -10.5921]])
0 : 0.0141
1 : 0.0001
2 : 0.5225
3 : 0.4481
4 : 0.0001
5 : 0.0000
6 : 0.0016
7 : 0.0000
8 : 0.0136
9 : 0.0000
预测结果: 2
